In [1]:
import random
import pickle

import numpy as np
import pandas as pd
from nltk.tokenize import RegexpTokenizer

from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import LSTM, Dense, Activation
from tensorflow.keras.optimizers import RMSprop

In [2]:
text_data = pd.read_csv("Text-Gen/fake_or_real.csv")

In [3]:
text = list(text_data.text.values)
combined_text = " ".join(text)

In [4]:
partial_text = combined_text[:10000]

In [5]:
tokenizer = RegexpTokenizer(r"\w+")
tokens = tokenizer.tokenize(partial_text.lower())

In [6]:
unique_tokens = np.unique(tokens)
unique_token_index = {token : idx for idx, token in enumerate(unique_tokens)}

In [7]:
initial_words = 10
input = []
next_words = []

for i in range(len(tokens) - initial_words):
    input.append(tokens[i:i + initial_words])
    next_words.append(tokens[i + initial_words])

In [8]:
X = np.zeros((len(input), initial_words, len(unique_tokens)), dtype=bool)
y = np.zeros((len(next_words), len(unique_tokens)), dtype=bool)

In [9]:
for i, words in enumerate(input):
    for j, word in enumerate(words):
        X[i, j, unique_token_index[word]] = 1
    y[i, unique_token_index[next_words[i]]] = 1

In [10]:
model = Sequential()
model.add(LSTM(128, input_shape=(initial_words, len(unique_tokens)), return_sequences=True))
model.add(LSTM(128))
model.add(Dense(len(unique_tokens)))
model.add(Activation("softmax"))

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [11]:
model.compile(loss="categorical_crossentropy", optimizer=RMSprop(learning_rate=0.01), metrics = ["accuracy"])
model.fit(X, y, batch_size=128, epochs = 30, shuffle=True)

Epoch 1/30
14/14 ━━━━━━━━━━━━━━━━━━━━ 1s 21ms/step - accuracy: 0.0398 - loss: 6.3113 
Epoch 2/30
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - accuracy: 0.0620 - loss: 5.8205
Epoch 3/30
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - accuracy: 0.0723 - loss: 5.7085
Epoch 4/30
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - accuracy: 0.0658 - loss: 5.7260
Epoch 5/30
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - accuracy: 0.0606 - loss: 5.7507
Epoch 6/30
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - accuracy: 0.0600 - loss: 5.6693
Epoch 7/30
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - accuracy: 0.0649 - loss: 5.5991
Epoch 8/30
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - accuracy: 0.0585 - loss: 5.6085
Epoch 9/30
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - accuracy: 0.0523 - loss: 5.5013
Epoch 10/30
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - accuracy: 0.0730 - loss: 5.2874
Epoch 11/30
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - accuracy: 0.0735 - loss: 5.1423
Epoch 12/30
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - accuracy

In [12]:
model.save("text_gen_model.h5")

In [13]:
model = load_model("text_gen_model.h5")

In [14]:
def predict_next_word(input, best):
    input = input.lower()
    X = np.zeros((1, initial_words, len(unique_tokens)))
    for i, word in enumerate(input.split()):
        X[0, i, unique_token_index[word]] = 1

    predictions = model.predict(X)[0]
    return np.argpartition(predictions, -best)[-best:]

In [15]:
possible = predict_next_word("He will have to look into this thing and he", 5)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 87ms/step


In [16]:
print([unique_tokens[idx] for idx in possible])

['really', 'many', 'gone', 'not', 'be']


In [17]:
def generate_text(input, length, choices=3):
    sequence = input.split()
    current_pos = 0
    for i in range(length):
        sub_sequence = " ".join(tokenizer.tokenize(" ".join(sequence).lower())[current_pos:current_pos+initial_words])
        try:
            choice = unique_tokens[random.choice(predict_next_word(sub_sequence, creativity))]
        except:
            choice = random.choice(unique_tokens)
        sequence.append(choice)
        current_pos+=1
    return " ".join(sequence)
    

In [19]:
generate_text("I want to see what is possible in this world", 20, 5)

'I want to see what is possible in this world circulated spinmeisters re thought them up film final unapologetic https paranoid their careers very rating went pro away director make'